# Create Submission From Model

Run inference on the test images to create results ready for submission. This will create a folder called `submission` in the current directory.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch, torchvision
torch.__version__

In [ ]:
from detectron2.evaluation.xview_evaluation import *

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import os
import numpy as np
import json
import shutil
import cv2
import random
import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Set some parameters.
LOCALIZATION_SUBMISSION_FOLDER = "SUBMISSION_LOCALIZATION"
# LOCALIZATION_MODEL_CONFIG = "./configs/xview/localization-28.yaml"
DAMAGE_SUBMISSION_FOLDER = "SUBMISSION_DAMAGE"
DAMAGE_MODEL_CONFIG = "./configs/xview/joint-11.yaml"

In [ ]:
# Removes and creates folders (clears them).
shutil.rmtree(LOCALIZATION_SUBMISSION_FOLDER, ignore_errors=True)
shutil.rmtree(DAMAGE_SUBMISSION_FOLDER, ignore_errors=True)
os.mkdir(LOCALIZATION_SUBMISSION_FOLDER)
os.mkdir(DAMAGE_SUBMISSION_FOLDER)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# # Load localization model.
# cfg_localization = get_cfg()
# cfg_localization.merge_from_file(LOCALIZATION_MODEL_CONFIG)
# # Load localization checkpoint.
# cfg_localization.MODEL.WEIGHTS = os.path.join(cfg_localization.OUTPUT_DIR, "model_0159999.pth")
# # cfg_localization.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
# predictor_localization = DefaultPredictor(cfg_localization)

# Load damage model.
cfg_damage = get_cfg()
cfg_damage.merge_from_file(DAMAGE_MODEL_CONFIG)
# Load damage checkpoint.
cfg_damage.MODEL.WEIGHTS = os.path.join(cfg_damage.OUTPUT_DIR, "model_0169999.pth")
# cfg_damage.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
predictor_damage = DefaultPredictor(cfg_damage)

In [ ]:
image_filenames = sorted(glob.glob("data/test_images_quad/*"))
count = 0
bl = None
br = None
tl = None
tr = None
temp = np.zeros((1024, 1024)).astype(int)
for filename in image_filenames:
    
    mod_val = count % 4
    
    # Choose the model type based on pre or post localization
    if filename.find("pre") >= 0:
        pass
#         # LOCALIZATION model.
#         outputs = predictor_localization(image)
    
#         output = outputs["sem_seg"].argmax(dim=0).cpu()
#         pred_image = np.array(output, dtype=np.int)
        
#         if mod_val == 0:
#             bl = pred_image
#         elif mod_val == 1:
#             br = pred_image
#         elif mod_val == 2:
#             tl = pred_image
#         elif mod_val == 3:
#             tr = pred_image
            
#             temp[0:512,0:512] = tl
#             temp[0:512,512:1024] = tr
#             temp[512:1024,0:512] = bl
#             temp[512:1024,512:1024] = br
        
#             name = os.path.basename(filename).replace("_tr", "")
#             new_filename = name.replace("pre", "localization")[:-4] + "_prediction.png"
#             submission_filename = os.path.join(LOCALIZATION_SUBMISSION_FOLDER, new_filename)

#             print(submission_filename)
#             cv2.imwrite(submission_filename, temp)
        
    elif filename.find("post") >= 0:
        # Load the image.
        image = cv2.imread(filename)
        pre_image = cv2.imread(filename.replace("post", "pre"))
    
        # DAMAGE model.
        outputs = predictor_damage(image, pre_image)
    
        output = outputs["sem_seg"].argmax(dim=0).cpu()
        pred_image = np.array(output, dtype=np.int)
        
        if mod_val == 0:
            bl = pred_image
        elif mod_val == 1:
            br = pred_image
        elif mod_val == 2:
            tl = pred_image
        elif mod_val == 3:
            tr = pred_image
            
            temp[0:512,0:512] = tl
            temp[0:512,512:1024] = tr
            temp[512:1024,0:512] = bl
            temp[512:1024,512:1024] = br
        
            name = os.path.basename(filename).replace("_tr", "")
            new_filename = name.replace("post", "damage")[:-4] + "_prediction.png"
            submission_filename = os.path.join(DAMAGE_SUBMISSION_FOLDER, new_filename)

            print(submission_filename)
            cv2.imwrite(submission_filename, temp)
            
            new_filename = name.replace("post", "localization")[:-4] + "_prediction.png"
            submission_filename = os.path.join(LOCALIZATION_SUBMISSION_FOLDER, new_filename)

            print(submission_filename)
            cv2.imwrite(submission_filename, temp)
        
    count += 1


In [ ]:
import datetime
now = datetime.datetime.now()
this_time = now.strftime("%m_%d_%Y_%H_%M_%S")
submission_folder = "submission_{}".format(this_time)
print(submission_folder)
os.mkdir(submission_folder)

In [ ]:
import os
os.system("cp {}/* {}/".format(LOCALIZATION_SUBMISSION_FOLDER, submission_folder))
os.system("cp {}/* {}/".format(DAMAGE_SUBMISSION_FOLDER, submission_folder))
os.system("zip -r {}.zip {}".format(submission_folder, submission_folder))